Kuliah dari : Dr Suryasatriya Trihandaru,M.Sc.nat 

In [32]:
corpus = """
Saya suka makan sayur bayam.
Saya suka 3 jagung rebus!
Sop kacang merah enak sekali.
Ayam goreng tidak suka.
Bebek bakar banyak lemak.
"""

In [35]:
kalimat = corpus.replace('.','').replace('3','').replace('!',' ').lower().split("\n")[1:-1]
kalimat

['saya suka makan sayur bayam',
 'saya suka  jagung rebus ',
 'sop kacang merah enak sekali',
 'ayam goreng tidak suka',
 'bebek bakar banyak lemak']

In [37]:
sentimen = [1, 1, 1, 0, 0]

In [38]:
kalimat[0].split()

['saya', 'suka', 'makan', 'sayur', 'bayam']

In [39]:
token = [k.split() for k in kalimat]
(token)

[['saya', 'suka', 'makan', 'sayur', 'bayam'],
 ['saya', 'suka', 'jagung', 'rebus'],
 ['sop', 'kacang', 'merah', 'enak', 'sekali'],
 ['ayam', 'goreng', 'tidak', 'suka'],
 ['bebek', 'bakar', 'banyak', 'lemak']]

In [40]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [41]:
tok = Tokenizer()
tok.fit_on_texts(kalimat)
tok.word_index

{'suka': 1,
 'saya': 2,
 'makan': 3,
 'sayur': 4,
 'bayam': 5,
 'jagung': 6,
 'rebus': 7,
 'sop': 8,
 'kacang': 9,
 'merah': 10,
 'enak': 11,
 'sekali': 12,
 'ayam': 13,
 'goreng': 14,
 'tidak': 15,
 'bebek': 16,
 'bakar': 17,
 'banyak': 18,
 'lemak': 19}

In [44]:
tok.index_word

{1: 'suka',
 2: 'saya',
 3: 'makan',
 4: 'sayur',
 5: 'bayam',
 6: 'jagung',
 7: 'rebus',
 8: 'sop',
 9: 'kacang',
 10: 'merah',
 11: 'enak',
 12: 'sekali',
 13: 'ayam',
 14: 'goreng',
 15: 'tidak',
 16: 'bebek',
 17: 'bakar',
 18: 'banyak',
 19: 'lemak'}

In [45]:
tok.word_counts

OrderedDict([('saya', 2),
             ('suka', 3),
             ('makan', 1),
             ('sayur', 1),
             ('bayam', 1),
             ('jagung', 1),
             ('rebus', 1),
             ('sop', 1),
             ('kacang', 1),
             ('merah', 1),
             ('enak', 1),
             ('sekali', 1),
             ('ayam', 1),
             ('goreng', 1),
             ('tidak', 1),
             ('bebek', 1),
             ('bakar', 1),
             ('banyak', 1),
             ('lemak', 1)])

In [46]:
print(kalimat[0])
for t in token[0]:
    print(t, tok.word_index[t], end=' ')

saya suka makan sayur bayam
saya 2 suka 1 makan 3 sayur 4 bayam 5 

In [47]:
#untuk buat sequences 
sequences = tok.texts_to_sequences(kalimat)
sequences 

[[2, 1, 3, 4, 5],
 [2, 1, 6, 7],
 [8, 9, 10, 11, 12],
 [13, 14, 15, 1],
 [16, 17, 18, 19]]

In [48]:
[len(s) for s in sequences]

[5, 4, 5, 4, 4]

In [49]:
#Panjang Kalimat disamakan dengan padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

maxlen = 5 

pad_seq = pad_sequences(sequences, maxlen=maxlen, padding='post')
pad_seq

array([[ 2,  1,  3,  4,  5],
       [ 2,  1,  6,  7,  0],
       [ 8,  9, 10, 11, 12],
       [13, 14, 15,  1,  0],
       [16, 17, 18, 19,  0]], dtype=int32)

In [50]:
tok.index_word.get(0, 'PAD-0')

'PAD-0'

In [51]:
for i in range(len(tok.index_word)):
    print (tok.index_word.get(i, 'PAD-0'))

PAD-0
suka
saya
makan
sayur
bayam
jagung
rebus
sop
kacang
merah
enak
sekali
ayam
goreng
tidak
bebek
bakar
banyak


Carilah vektor yang merepresentasikan masing masing kata
carilah data KTP (Angka saja) untuk setiap nama orang
Caranya : buatlah analisa sentimen pada corpus

In [52]:
import numpy as np

X = np.array(pad_seq)
y = np.array(sentimen)
X, y

(array([[ 2,  1,  3,  4,  5],
        [ 2,  1,  6,  7,  0],
        [ 8,  9, 10, 11, 12],
        [13, 14, 15,  1,  0],
        [16, 17, 18, 19,  0]], dtype=int32),
 array([1, 1, 1, 0, 0]))

Buatlah arsitek neural network yang menghubungkan X dan y -> Klasifikasi

In [53]:
#buatlah artificial neural network untuk klasifikasi X versus y, dimana y mempunyai 2 opsi  saja (binary) dan tuliskan dalam functional-API keras

from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense

#mendefinisikan inputan layer
input_layer = Input(shape=(maxlen,))

#embedding layer untuk merepresentasikan words sebagai vektors
embedding_layer = Embedding(input_dim=len(tok.word_index) + 1, output_dim=4)(input_layer)

#flatten the embedding output
flatten_Layer = Flatten() (embedding_layer)

#Dense layer for clasification
dense_layer = Dense(10, activation='relu')(flatten_Layer)

#output layer with sigmoid activation for binary classification
output_layer = Dense(1, activation='sigmoid')(dense_layer)

#create the model
model = Model(inputs=input_layer, outputs=output_layer)

#compile the model 
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#print the model summary
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 5)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 5, 4)           │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 301 (1.18 KB)

 Trainable params: 301 (1.18 KB)

 Non-trainable params: 0 (0.00 B)

In [54]:
embedding_layer.shape

(None, 5, 4)

In [55]:
hasil = model.fit (X, y, epochs=100, verbose=1)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.6000 - loss: 0.6937
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.6000 - loss: 0.6911
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.6000 - loss: 0.6885
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.8000 - loss: 0.6860
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.8000 - loss: 0.6834
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.8000 - loss: 0.6808
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.8000 - loss: 0.6782
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 1.0000 - loss: 0.6756
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 1.0000 - loss: 0.6730
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 1.0000 - loss: 0.6705
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 1.0000 - loss: 0.6680
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 1.0000 - loss

In [56]:
for i, v in enumerate(model.layers[1].get_weights()[0]):
    print(tok.index_word.get(i, 'PAD-0'), v)

PAD-0 [-0.10307678 -0.1584892   0.10459942 -0.1152449 ]
suka [-0.16568762 -0.1930917   0.06095291  0.07968808]
saya [-0.09342223  0.13586532 -0.14124036 -0.06922799]
makan [ 0.08806515 -0.09579623  0.01012499 -0.15752183]
sayur [ 0.13829729  0.09671169  0.06818481 -0.07528894]
bayam [ 0.12572649 -0.06996195 -0.08305185 -0.10063075]
jagung [ 0.11419563 -0.16659662  0.0169486  -0.13775739]
rebus [ 0.06021826  0.16562234  0.11896593 -0.10473663]
sop [-0.12797458  0.10980434 -0.09144661 -0.11297892]
kacang [-0.12394655 -0.09309578  0.13402946 -0.0091818 ]
merah [ 0.12763584 -0.08420504 -0.01678133 -0.1330657 ]
enak [ 0.10802402  0.16201152  0.07915835 -0.14478754]
sekali [ 0.09317629 -0.12013874  0.01033682 -0.15302528]
ayam [ 0.12905    -0.03130238  0.08531202 -0.05698452]
goreng [0.05393425 0.09178773 0.11646352 0.10467695]
tidak [-0.06270448 -0.06876616 -0.09333988  0.08411887]
bebek [ 0.08602329  0.13860743  0.14000317 -0.07500389]
bakar [ 0.14622658  0.06081134  0.14536074 -0.11517438

In [69]:
# Hitung kata yang mirip memakai cosine similarity

from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def get_cosine_similarity(word1, word2, embedding_matrix):
    """
      Calculates the cosine similarity between two words using their embeddings.

  Args:
    word1: The first word.
    word2: The second word.
    embedding_matrix: The matrix containing the word embeddings.

  Returns:
    The cosine similarity between the two words.
    """

    word1_index = tok.word_index.get(word1)
    word2_index = tok.word_index.get(word2)

    if word1_index is None or word2_index is None : 
        return 0
    
    word1_embedding = embedding_matrix[word1_index - 1]
    word2_embedding = embedding_matrix[word2_index - 1]

    return cosine_similarity([word1_embedding], [word2_embedding])[0][0]

#assuming we have an embedding matrix from our model
embedding_matrix = model.layers[1].get_weights()[0]

#contoh 
word1 = "ayam"
word2 = "bebek"

similarity = get_cosine_similarity(word1, word2, embedding_matrix)
print(f"Cosine similarity between '{word1}' and '{word2}': {similarity} ")



Cosine similarity between 'ayam' and 'bebek': -0.3381393551826477 


In [70]:
for w in tok.word_index:
    s=get_cosine_similarity("ayam", w, embedding_matrix)
    print(w, tok.word_index[w], s)

suka 1 0.5323514
saya 2 -0.06441361
makan 3 -0.3229616
sayur 4 0.9942373
bayam 5 0.3172058
jagung 6 0.82630473
rebus 7 0.9808262
sop 8 0.058028683
kacang 9 -0.18246663
merah 10 0.054763887
enak 11 0.9606764
sekali 12 0.24629039
ayam 13 1.0
goreng 14 0.70053166
tidak 15 -0.50904286
bebek 16 -0.33813936
bakar 17 0.0872055
banyak 18 0.4831502
lemak 19 -0.47538996


In [73]:
#Membuat prediksi untuk kalimat "saya makan ayam dan bebek"

def predict_sentiment (text, tok, model, maxlen):

    """
  Predicts the sentiment of a given text using the trained model.

  Args:
    text: The input text.
    tok: The tokenizer used for the model.
    model: The trained sentiment analysis model.
    maxlen: The maximum length of the input sequence.

  Returns:
    The predicted sentiment (1 for positive, 0 for negative).
  """

    sequences = tok.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequences, maxlen=maxlen, padding='post')
    prediction = model.predict(padded_sequence)[0][0]
    return 1 if prediction >= 0.5 else 0

new_text = "Saya makan sop bayam"
predicted_sentiment = predict_sentiment(new_text, tok, model, maxlen)
print (f"Predicted sentiment for '{new_text}': {predicted_sentiment} ")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Predicted sentiment for 'Saya makan sop bayam': 1 


In [77]:
# Puisi oleh ChatGPT
puisi = """
Lukisan Alam dalam Cahaya Senja
Di ufuk barat mentari perlahan turun ke peraduan,
Langit jingga membentang, menari bersama awan perak.
Angin berbisik lembut di sela dedaunan hijau,
Menyanyikan lagu sunyi bagi jiwa yang tenang.
Burung-burung pulang melintas di langit keemasan,
Sayapnya menggores cahaya, lukisan tanpa batas.
Gunung-gunung berdiri tegap dalam bayang senja,
Berselimut kabut tipis yang melayang perlahan.
Sungai berkilau membelah hamparan lembah nan luas,
Airnya membisikkan kisah kepada batu dan pasir.
Bunga-bunga mekar di ladang yang tak bertepi,
Warnanya berpendar dalam cahaya yang meredup.
Di kejauhan sawah hijau melambai gemulai,
Riak airnya memantulkan warna jingga merona.
Petani tersenyum memandang hasil jerih payah,
Bersyukur pada bumi yang terus memberi.
Laut membentang luas, biru bagai kaca bening,
Ombak kecil berkejaran mencumbu pasir putih.
Pohon kelapa menari diterpa angin lembut,
Membawa harum garam dari samudra lepas.
Bintang mulai muncul, menghiasi langit gelap,
Menjadi lentera bagi malam yang hening.
Rembulan tersenyum malu di balik awan,
Menyapa bumi dengan sinarnya yang lembut.
Di bawah cahaya kuning lampu desa kecil,
Anak-anak berlarian membawa tawa ceria.
Nyanyian jangkrik berpadu dalam harmoni malam,
Mengisi kesunyian dengan irama alam.
Dunia beristirahat dalam dekapan malam syahdu,
Alam bernapas tenang dalam kedamaian abadi.
Esok pagi mentari akan bersinar kembali,
Membangunkan keindahan dari tidur yang damai.
"""
puisi = puisi.lower().replace('.','').replace(',',' ').replace('\n',' ')
print(puisi ) 

 lukisan alam dalam cahaya senja di ufuk barat mentari perlahan turun ke peraduan  langit jingga membentang  menari bersama awan perak angin berbisik lembut di sela dedaunan hijau  menyanyikan lagu sunyi bagi jiwa yang tenang burung-burung pulang melintas di langit keemasan  sayapnya menggores cahaya  lukisan tanpa batas gunung-gunung berdiri tegap dalam bayang senja  berselimut kabut tipis yang melayang perlahan sungai berkilau membelah hamparan lembah nan luas  airnya membisikkan kisah kepada batu dan pasir bunga-bunga mekar di ladang yang tak bertepi  warnanya berpendar dalam cahaya yang meredup di kejauhan sawah hijau melambai gemulai  riak airnya memantulkan warna jingga merona petani tersenyum memandang hasil jerih payah  bersyukur pada bumi yang terus memberi laut membentang luas  biru bagai kaca bening  ombak kecil berkejaran mencumbu pasir putih pohon kelapa menari diterpa angin lembut  membawa harum garam dari samudra lepas bintang mulai muncul  menghiasi langit gelap  menjad

In [89]:
tok = Tokenizer()
tok.fit_on_texts([puisi])

In [90]:
print ("panjang index adalah : " , len(tok.index_word))

panjang index adalah :  155


In [91]:
tok.index_word[155]

'damai'

In [92]:
sequences = tok.texts_to_sequences([puisi])[0]
sequences[0:20]

[9, 5, 3, 4, 10, 2, 34, 35, 11, 12, 36, 37, 38, 6, 13, 14, 15, 39, 16, 40]

In [94]:
X = []
y = []
window = 5 #merupakan banyaknya kata max dalam sebuah kalimat
for i in range(len(sequences)-window):
    X.append(sequences[i:i+window])
    y.append(sequences[i+window])

X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)

(204, 5) (204,)


In [95]:
#Buatkan LSTM untuk X, dan y
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Embedding, LSTM, Dense

#Definisikan model LSTM
model = Sequential()
model.add(Embedding(len(tok.word_index)+1, 128, input_length=window))
model.add(LSTM(128))
model.add(Dense(len(tok.word_index) + 1, activation='softmax'))

#model di compile 
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#train the model
model.fit(X, y, epochs=100, verbose=1)


d:\KULIAH\S2\Semester 2\Large Language Model\Kuliah dengan Pak Surya\Kodingan LLM\env\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.0027 - loss: 5.0507   
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1475 - loss: 5.0367 
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1774 - loss: 5.0216
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1575 - loss: 5.0062 
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1447 - loss: 4.9856 
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1185 - loss: 4.9558 
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1087 - loss: 4.8904 
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.0707 - loss: 4.7601   
Epoch 9/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0958 - loss: 4.6036 
Epoch 10/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1613 - loss: 4.3635 
Epoch 11/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.2156 - loss: 4.2650
Epoch 12/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2000

In [100]:
#lanjutkan untuk prediksi kalimat ini : "Nyanyian Jangkrik berpadu dalam Harmoni"

def generate_next_word(seed_text, model, tok, max_sequences_len, num_words_to_generate=1):
    for _ in range(num_words_to_generate):
        token_list = tok.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequences_len, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tok.word_index.items():
            if index == predicted:
                output_word = word
                break 
            seed_text += " " + output_word
    return  seed_text

seed_text = "angin malam ini membuat aku kedinginan"
next_word = generate_next_word(seed_text, model, tok, window)
next_word

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


'angin malam ini membuat aku kedinginan             '

Kuliah selanjutnya adalah Membuat Prediksi dengan MultiheadAttention Neural Network (Sebagai Kunci LLM)

In [101]:
# Presensi dan Kirim jawaban
import requests
import json
def kirimjawaban(nama, nim, jawab):
    url="https://script.google.com/macros/s/AKfycbwcPuGPKy5NxQShEttnSmPkLVksCI7IREHXH-oQ4OlaW55LP3KtkKQDReIi3ZsgkT7cxQ/exec"
    data = {"nama": nama,"nomor_induk": nim,"jawab": jawab}
    print(f"Nama: {nama}")
    print(f"NIM: {nim}")
    print(f"Jawaban: {jawab}")
    kirim = input("Kirim Jawaban? (y/n)")
    if kirim == 'y':
      headers = {"Content-Type": "application/json"}
      response = requests.post(url, data=json.dumps(data), headers=headers)
      print(response.text)
    else:
      print("Jawaban tidak dikirim")

In [ ]:
# Ubah nama dan nim
nama = "Harfely Leipary"
nim ="632024001"
jawab=['Hadir Pertemuan 1']
kirimjawaban(nama, nim, jawab)